<a href="https://colab.research.google.com/github/MattJCR/Practica-DEEP-LEARNING/blob/master/Practica_DEEP_LEARNING_Parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PR4.2. DEEP LEARNING

## Parte 1:
Seguir el ejemplo de clasificación de imágenes de flores tratado en los apuntes. Generar el colab y
probar su funcionamiento.

In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
import tensorflow as tf

Descargamos los datos de entrenamiento

In [ ]:
drive.mount('/content/drive')
!unzip -n '/content/drive/My Drive/DL_flowers.zip' >> /dev/null
data_path = 'flowers/'
imgs_path = data_path + "images/"
labels_path = data_path + "labels.csv"
labels = pd.read_csv(labels_path, dtype = {"class":"category"})

Mounted at /content/drive


Particionamos los datos de entrenamiento, validación y test

In [ ]:
seed = 0
train_data, test_data = train_test_split(labels, test_size=0.3, random_state = seed)
dev_data, test_data = train_test_split(labels, test_size=0.5, random_state = seed)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
dev_data = dev_data.reset_index(drop=True)

Preparacion de los datos

In [ ]:
# Normalización de los datos utilizando reescale
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
classes = ["0","1","2","3","4"] # Clases del problema
n_classes = len(classes) # Numero de clases
img_width = img_height = 224 # Dimensiones de la imagen
x_col = "image_name"
y_col = "class"

Generacion del batch con los datos para las 3 particiones

In [ ]:
batch_size = 128
train_generator = datagen.flow_from_dataframe(
    dataframe = train_data, directory = imgs_path, x_col = x_col, y_col= y_col, has_ex = True, class_mode = "categorical", target_size = (img_width,img_height), batch_size = batch_size, classes = classes
)

test_generator = datagen.flow_from_dataframe(
    dataframe = test_data, directory = imgs_path, x_col = x_col, y_col= y_col, has_ex = True, class_mode = "categorical", target_size = (img_width,img_height), batch_size = batch_size, classes = classes
)

dev_generator = datagen.flow_from_dataframe(
    dataframe = dev_data, directory = imgs_path, x_col = x_col, y_col= y_col, has_ex = True, class_mode = "categorical", target_size = (img_width,img_height), batch_size = batch_size, classes = classes
)

Found 3020 validated image filenames belonging to 5 classes.
Found 2158 validated image filenames belonging to 5 classes.
Found 2157 validated image filenames belonging to 5 classes.


Cargamos el modelo InceptionV3 pre-entrenado con ImageNet

In [ ]:
base_model = tf.keras.applications.InceptionV3(weights = "imagenet", include_top = False, input_shape = (img_width,img_height, 3))

87910968/87910968 [==============================] - 1s 0us/step


Ajustamos las capas del modelo ya preeentrenadas para que no se actualicen (entrenen nuevamente). Solamente entrenaremos las capas nuevas que añadimos.

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

Añadimos nuevas capas al final para adaptar el modelo a nustro problema y lo creamos.

In [ ]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation = "relu")(x)

# Añadimos la ultima capa softmax con el numero de clases para la salida de la red.
predictions = tf.keras.layers.Dense(n_classes, activation = "softmax")(x)

In [ ]:
# Creamos el modelo y lo compilamos
model = tf.keras.models.Model(inputs = [base_model.input], outputs = [predictions])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),
    metrics = ["accuracy"]
)

Entrenamos el modelo

In [ ]:
model.fit(
    train_generator,
    epochs = 10,
    verbose = 2,
    steps_per_epoch = len(train_data) / batch_size,
    validation_data = dev_generator,
    validation_steps = len(dev_data) / batch_size,
    use_multiprocessing = True
)

Epoch 1/10
23/23 - 643s - loss: 1.5468 - accuracy: 0.3321 - val_loss: 1.3284 - val_accuracy: 0.4766 - 643s/epoch - 27s/step
Epoch 2/10
23/23 - 629s - loss: 1.1867 - accuracy: 0.5642 - val_loss: 1.0595 - val_accuracy: 0.6351 - 629s/epoch - 27s/step
Epoch 3/10
23/23 - 682s - loss: 0.9801 - accuracy: 0.6732 - val_loss: 0.9003 - val_accuracy: 0.7010 - 682s/epoch - 29s/step
Epoch 4/10
23/23 - 684s - loss: 0.8535 - accuracy: 0.7219 - val_loss: 0.7994 - val_accuracy: 0.7344 - 684s/epoch - 29s/step
Epoch 5/10
23/23 - 626s - loss: 0.7679 - accuracy: 0.7503 - val_loss: 0.7259 - val_accuracy: 0.7520 - 626s/epoch - 27s/step
Epoch 6/10
23/23 - 622s - loss: 0.7045 - accuracy: 0.7705 - val_loss: 0.6717 - val_accuracy: 0.7789 - 622s/epoch - 26s/step
Epoch 7/10
23/23 - 631s - loss: 0.6562 - accuracy: 0.7868 - val_loss: 0.6296 - val_accuracy: 0.7854 - 631s/epoch - 27s/step
Epoch 8/10
23/23 - 684s - loss: 0.6173 - accuracy: 0.7983 - val_loss: 0.5932 - val_accuracy: 0.7997 - 684s/epoch - 29s/step
Epoch 9/

In [ ]:
test_loss, test_acc = model.evaluate(
    test_generator,
    steps = len(test_data) / batch_size,
    verbose = 1,
    use_multiprocessing = True
)
print("test_loss: %.4f, test_acc %.4f" % (test_loss, test_acc))

16/16 [==============================] - 265s 15s/step - loss: 0.5782 - accuracy: 0.8072
test_loss: 0.5782, test_acc 0.8072
